In [1]:
import sqlite3
import pandas as pd
from tabulate import tabulate
from contextlib import contextmanager
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S', 
    handlers=[
        logging.FileHandler("score_rh.log"),  # Salva logs em um arquivo
        logging.StreamHandler()  # Exibe logs no console
    ]
)

DB_PATH = "/app/data/teste___desafio_técnico_-_analista_de_dados_pleno_anexo_db.db"
SQL_SCRIPT_PATH = "/app/scripts/data_prep.sql"

In [2]:
@contextmanager
def conectar_banco():
    """Gerenciador de contexto para conexão com o banco de dados."""
    conn = None
    try:
        conn = sqlite3.connect(DB_PATH)
        yield conn  # Retorna a conexão para ser usada no bloco `with`
    except Exception as e:
        logging.error(f"Erro ao conectar ao banco de dados: {e}")
        raise
    finally:
        if conn:
            conn.close()  # Fecha a conexão após o uso

In [ ]:
def listar_tabelas():
    """Lista todas as tabelas do banco SQLite."""
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tabelas = cursor.fetchall()
    
    conn.close()
    
    return [t[0] for t in tabelas]

def visualizar_tabela(nome_tabela, haslimit=True, limite=5):
    try:
        with conectar_banco() as conn:
            if haslimit:
                query = f"SELECT * FROM {nome_tabela} LIMIT {limite};"
            else:
                query = f"SELECT * FROM {nome_tabela};"
            df = pd.read_sql_query(query, conn)
        return df
    except Exception as e:
        logging.error(f"Erro ao visualizar tabela {nome_tabela}: {e}")
        return pd.DataFrame()


tabelas = listar_tabelas()
print(f"Tabelas encontradas: {tabelas}")

# Mostrar 5 registros de cada tabela
for tabela in tabelas:
    print(f"\nAmostra da tabela: {tabela}")
    print(tabulate(visualizar_tabela(tabela, limite=5), headers= "keys", tablefmt="grid"))


Tabelas encontradas: ['competencias', 'vagas', 'candidatos', 'vagacompetencia', 'candidato_vaga', 'experiencias', 'competencia_experiencia']

Amostra da tabela: competencias
+----+------+---------------+--------------+---------------------------------------------------+------------+
|    |   id | nome          | tipo         | descricao                                         | area       |
+====+======+===============+==============+===================================================+============+
|  0 |    1 | competencia 1 | Conhecimento | Conhecimento importante para a área Financeira    | Financeira |
+----+------+---------------+--------------+---------------------------------------------------+------------+
|  1 |    2 | competencia 2 | Atitude      | Atitude importante para a área Comercial          | Comercial  |
+----+------+---------------+--------------+---------------------------------------------------+------------+
|  2 |    3 | competencia 3 | Atitude      | Atitude imp

In [7]:
df = visualizar_tabela('vagas', haslimit=False)


In [9]:
df['departamento'].drop_duplicates()

0     Contabilidade
1        Financeira
2        Engenharia
3                RH
7             Dados
15        Comercial
Name: departamento, dtype: object